# IMDB Movie Reviews: Sentiment Analysis

In [2]:
import numpy as np
import spacy
import os.path
import glob
import random

#Dataset: https://www.kaggle.com/iarunava/imdb-movie-reviews-dataset

filenames = glob.glob(os.path.join(os.getcwd(),"/Users/Ben/Desktop/Vital Strategies/Datasets/Imdb/train/neg/*.txt"))
posFilenames = glob.glob(os.path.join(os.getcwd(),"/Users/Ben/Desktop/Vital Strategies/Datasets/Imdb/train/pos/*.txt"))

for p in posFilenames:
    filenames.append(p)

In [9]:
nlp = spacy.load("en_core_web_lg")

In [21]:
nlp = spacy.load("en_core_web_lg")

textcat = nlp.create_pipe(
            "textcat",
            config={
                "exclusive_classes": True,
                "architecture": "simple_cnn",
            }
        )
nlp.add_pipe(textcat, last=True)

textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

1

In [41]:
#FORMAT: train_data = [(u"Uber blew through $1 million", {"entities": [(0, 4, "ORG")]})]

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    
    len_of_data = len(filenames)
    
    idxs = list(range(len_of_data))
    
    ITERATIONS = 2
    for i in range(ITERATIONS):
        print("epoch: ", i, "/", ITERATIONS)
        random.shuffle(idxs)
        
        
        #Batches:
        SIZE = 100
        if(len_of_data % SIZE != 0):
            print("BAD BATCH SIZE!")
        for batch in range(int(len_of_data/SIZE)):
            print("  batch: ",batch , "/", int(len_of_data/SIZE))
            annotations = []
            texts = []
            for idx in idxs[batch*SIZE:SIZE*(batch+1)]:
                with open(filenames[idx],"r") as f:
                    texts.append(f.read())
                if(idx < len_of_data/2):
                    annotations.append({"cats":{"NEGATIVE":True, "POSITIVE":False}})
                else:
                    annotations.append({"cats":{"NEGATIVE":False, "POSITIVE":True}})
            nlp.update(texts, annotations, sgd=optimizer)
    
print("Training Complete! Saving model...")
nlp.to_disk(os.path.join(os.getcwd(),"/Users/Ben/Desktop/Vital Strategies/trained models/sentimentClassifier"))

epoch:  0 / 2
  batch:  0 / 250
  batch:  1 / 250
  batch:  2 / 250
  batch:  3 / 250
  batch:  4 / 250
  batch:  5 / 250
  batch:  6 / 250
  batch:  7 / 250
  batch:  8 / 250
  batch:  9 / 250
  batch:  10 / 250
  batch:  11 / 250
  batch:  12 / 250
  batch:  13 / 250
  batch:  14 / 250
  batch:  15 / 250
  batch:  16 / 250
  batch:  17 / 250
  batch:  18 / 250
  batch:  19 / 250
  batch:  20 / 250
  batch:  21 / 250
  batch:  22 / 250
  batch:  23 / 250
  batch:  24 / 250
  batch:  25 / 250
  batch:  26 / 250
  batch:  27 / 250
  batch:  28 / 250
  batch:  29 / 250
  batch:  30 / 250
  batch:  31 / 250
  batch:  32 / 250
  batch:  33 / 250
  batch:  34 / 250
  batch:  35 / 250
  batch:  36 / 250
  batch:  37 / 250
  batch:  38 / 250
  batch:  39 / 250
  batch:  40 / 250
  batch:  41 / 250
  batch:  42 / 250
  batch:  43 / 250
  batch:  44 / 250
  batch:  45 / 250
  batch:  46 / 250
  batch:  47 / 250
  batch:  48 / 250
  batch:  49 / 250
  batch:  50 / 250
  batch:  51 / 250
  batch:

  batch:  170 / 250
  batch:  171 / 250
  batch:  172 / 250
  batch:  173 / 250
  batch:  174 / 250
  batch:  175 / 250
  batch:  176 / 250
  batch:  177 / 250
  batch:  178 / 250
  batch:  179 / 250
  batch:  180 / 250
  batch:  181 / 250
  batch:  182 / 250
  batch:  183 / 250
  batch:  184 / 250
  batch:  185 / 250
  batch:  186 / 250
  batch:  187 / 250
  batch:  188 / 250
  batch:  189 / 250
  batch:  190 / 250
  batch:  191 / 250
  batch:  192 / 250
  batch:  193 / 250
  batch:  194 / 250
  batch:  195 / 250
  batch:  196 / 250
  batch:  197 / 250
  batch:  198 / 250
  batch:  199 / 250
  batch:  200 / 250
  batch:  201 / 250
  batch:  202 / 250
  batch:  203 / 250
  batch:  204 / 250
  batch:  205 / 250
  batch:  206 / 250
  batch:  207 / 250
  batch:  208 / 250
  batch:  209 / 250
  batch:  210 / 250
  batch:  211 / 250
  batch:  212 / 250
  batch:  213 / 250
  batch:  214 / 250
  batch:  215 / 250
  batch:  216 / 250
  batch:  217 / 250
  batch:  218 / 250
  batch:  219 / 250


## Load Model:

In [29]:
from spacy.lang.en import English
nlp = spacy.load((os.path.join(os.getcwd(),
                                       "/Users/Ben/Desktop/Vital Strategies/trained models/sentimentClassifier")))

In [30]:
# verify:
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

# Testing:

In [31]:
test_filenames = glob.glob(os.path.join(os.getcwd(),
                                        "/Users/Ben/Desktop/Vital Strategies/Datasets/Imdb/test/neg/*.txt"))
test_posFilenames = glob.glob(os.path.join(os.getcwd(),
                                           "/Users/Ben/Desktop/Vital Strategies/Datasets/Imdb/test/pos/*.txt"))


for p in test_posFilenames:
    test_filenames.append(p)

len_of_test = len(test_filenames)

In [50]:
print("Testing model...")
correct = 0
wrongs = []
for idx in range(len_of_test):
    
    if(idx%100 ==1):
        print("idx =",idx, "/tAccuracy =",correct/idx)
    
    with open(test_filenames[idx], "r") as f:
        doc = nlp(f.read())
    if(idx < len_of_test/2 and doc.cats["NEGATIVE"] > doc.cats["POSITIVE"] 
       or idx > len_of_test/2 and doc.cats["POSITIVE"] > doc.cats["NEGATIVE"]):
        correct += 1
    else:
        wrongs.append(idx)
    
print("Accuracy on Test set:", correct/len_of_test)  # Accuracy = .88


Testing model...
idx = 1  Accuracy = 1.0
idx = 101  Accuracy = 0.7920792079207921
idx = 201  Accuracy = 0.8407960199004975
idx = 301  Accuracy = 0.8438538205980066
idx = 401  Accuracy = 0.8628428927680798
idx = 501  Accuracy = 0.8702594810379242
idx = 601  Accuracy = 0.8635607321131448
idx = 701  Accuracy = 0.8601997146932953
idx = 801  Accuracy = 0.8614232209737828
idx = 901  Accuracy = 0.8501664816870145
idx = 1001  Accuracy = 0.8491508491508492
idx = 1101  Accuracy = 0.8510445049954587
idx = 1201  Accuracy = 0.8559533721898418
idx = 1301  Accuracy = 0.8562644119907763
idx = 1401  Accuracy = 0.8558172733761599
idx = 1501  Accuracy = 0.8534310459693538
idx = 1601  Accuracy = 0.8519675202998126
idx = 1701  Accuracy = 0.84891240446796
idx = 1801  Accuracy = 0.8489727928928373
idx = 1901  Accuracy = 0.8500789058390321
idx = 2001  Accuracy = 0.8525737131434283
idx = 2101  Accuracy = 0.8538791051880057
idx = 2201  Accuracy = 0.8518855065879146
idx = 2301  Accuracy = 0.8518035636679705
idx 

idx = 19401  Accuracy = 0.8750579867017164
idx = 19501  Accuracy = 0.8750833290600482
idx = 19601  Accuracy = 0.8750573950308658
idx = 19701  Accuracy = 0.8750317242779554
idx = 19801  Accuracy = 0.8751578203121054
idx = 19901  Accuracy = 0.8751821516506708
idx = 20001  Accuracy = 0.8751562421878906
idx = 20101  Accuracy = 0.8754788318989105
idx = 20201  Accuracy = 0.8754517103113707
idx = 20301  Accuracy = 0.8754741145756366
idx = 20401  Accuracy = 0.8757413852262144
idx = 20501  Accuracy = 0.8758109360518999
idx = 20601  Accuracy = 0.8759768943255182
idx = 20701  Accuracy = 0.875899714989614
idx = 20801  Accuracy = 0.8761598000096149
idx = 20901  Accuracy = 0.8763695516960911
idx = 21001  Accuracy = 0.8765296890624256
idx = 21101  Accuracy = 0.8766883086109664
idx = 21201  Accuracy = 0.8768454318192538
idx = 21301  Accuracy = 0.876249941317309
idx = 21401  Accuracy = 0.876501098079529
idx = 21501  Accuracy = 0.8767034091437608
idx = 21601  Accuracy = 0.8768112587380213
idx = 21701  A

NameError: name 'docs' is not defined

# Analyzing examples of incorrect predictions:

In [53]:
print("\nExamples of wrongs:")
random.shuffle(wrongs)
for i in range(10):
    with open(test_filenames[wrongs[i]],"r") as f:
        temp = f.read()
        print(wrongs[i], ":", temp)
        print(nlp(temp).cats,"\n")
    


Examples of wrongs:
21787 : Incredibly hilarious mid-70's Italian Rootsploitation with lots of non-consensual S&M, lesbian sex, gratuitous racial cruelty etc...Few redeeming cinematic qualities, except for the fairly cool theme music with dubby "African" drums and flute. Brilliant sample dialog: <br /><br />White Slave Owner (to White Plantation Manager): "You're so dumb, I'll bet you forgot to interrogate that n****r midwife!"<br /><br />White Plantation Manager: "Not only did I interrogate her, I did it so well she died before I could get any answers from her!"<br /><br />All the black actors have 70's afros, and say "yes, massa" in a high-pitched voice. The female lead has sex with everybody on the plantation. 10 Stars for fans of tasteless sleaze.
{'POSITIVE': 0.22175347805023193, 'NEGATIVE': 0.7782465219497681} 

23482 : People who say that some of what the Looney Tunes cartoons show is inappropriate for children have apparently forgotten something: they weren't originally intend

{'POSITIVE': 0.8238601088523865, 'NEGATIVE': 0.17613986134529114} 

18669 : What about DJ Cash Money??? This film fails in part by not covering the mid to late 80s. There was only a small mention of DJ Cheese in 86.<br /><br />Also, it's Grandmixer "DST", not "DXT"!!!!!
{'POSITIVE': 0.06487777084112167, 'NEGATIVE': 0.9351223111152649} 

22331 : Remember that friend in college who always insisted you rent the weirdest movie possible? This is the movie he would have made if he'd had the chance.<br /><br />I wish I could tell you exactly what Sea of Dust was about. It pretends to be the story of a doctor who gets sucked into weird goings on in the "Black Forest." He goes there to help, but ends up being caught between two young women, both of whom he seems to have a thing for. But that's just scratching the surface. This is the kind of movie where things just randomly happen...and not nice thing. People are constantly being whipped and stabbed. There's a pair of creepy little girls who ap

# Display:

In [50]:
import re

def save_vectors(vectors):
    with open(os.path.join(os.getcwd(), "/Users/Ben/Desktop/Vital Strategies/trained models/nlpVectors/Asgn2vectors.tsv"), "w") as f:
        for vector in vectors:
            for feature in vector:
                f.write(str(feature))
                f.write("\t")
                f.write("1.0\t1.0\t1.0\t") # BS so I can display on embedding projector
            f.write("\n")
            
    f.close()
def save_metadata(titles,labels):
    with open(os.path.join(os.getcwd(), "/Users/Ben/Desktop/Vital Strategies/trained models/nlpVectors/Asgn2metadata.tsv"), "w") as f:
        f.write("title\tcategory\n")
        for i, title in enumerate(titles):
            f.write(str(title))
            f.write("\t")
            f.write(labels[i])
            f.write("\n")
    f.close()

In [49]:
NUM_EXAMPLES_TO_SAVE = 1000

idxs = list(range(len_of_test))
random.shuffle(idxs)
idxs = idxs[:NUM_EXAMPLES_TO_SAVE]

vectors = [] #vectors in 1D - How POSTIVE it thinks a title is
titles = []
labels = []

for idx in idxs:
    with open(test_filenames[idx], "r") as f:
        temp = f.read()
        doc = nlp(temp)
        vectors.append([doc.cats["POSITIVE"]])
        titles.append(idx)
        if(idx < len_of_test/2 and doc.cats["NEGATIVE"] > doc.cats["POSITIVE"] ):
            labels.append("NEGATIVE")
        elif (idx > len_of_test/2 and doc.cats["POSITIVE"] > doc.cats["NEGATIVE"]):
            labels.append("POSTIVE")
        else:
            labels.append("INCORRECT")
save_vectors(vectors)
save_metadata(titles,labels)

TypeError: write() argument must be str, not int

In [52]:
def printatidx(idx):
    print("idx:",idx)
    with open(test_filenames[idx], "r") as f:
        print(f.read())

In [56]:
printatidx(14857)

idx: 14857
This movie was well done. It covers the difficulties a returning Vietnam veteran has in dealing with the horrors of war. Unfortunately the writers chose to focus on a Vet who had been involved in an act of atrocity. I was in Vietnam and only once heard of such an act by one who witnessed it. The offender was prosecuted and sentenced to many years in Leavenworth.<br /><br />The notion that only vets involved in atrocities had emotional problems is a disservice to all who served. All of the soldiers I knew personally or knew of by word of mouth were honorable soldiers who respected even the enemy and believed they were there to halt the spread of Communism. The biggest problem was coming home to learn that many Americans were opposed to the war. That is what caused many Veterans to feel they had taken part in something less than honorable. Not the manner in which they served.<br /><br />The ending depicted the father acting more as a belligerent bully than a loving, caring fat